In [2]:
#Import all the necessary libraries

import os
import hashlib
import pandas as pd
import numpy as np
import csv

#This will create a new CSV to store the username and password

file = open('customer.csv', 'w')
writer = csv.writer(file)
header = ['username', 'password']
writer.writerow(header)

file.close()

#This will create a new CSV to store the task and the status of the task

file1 = open("taskmanager.csv",'w', newline='')
writer = csv.writer(file1)
header=['username','task','status']
writer.writerow(header)

file.close()

# Using hashlib library we will hash the password at rest

def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

#Below is the code to check if username exists in the dataframe or not

def check_username(username, filename='customer.csv'):
    try:
        df = pd.read_csv(filename)
        return username in df["username"].values
    except FileNotFoundError as e:
        print(f"File not found :{e}")
        return False
    except Exception as e:
        print(f" Error checking username: {e}")
        return False

#The code below will register a new username password as row after checking for existing username. 
#The row then will be stored in the customer database
        
def register_username(username, password, filename='customer.csv'):
    if check_username(username, filename):
        print(f"Username: {username}")
        return
    hashed_password = hash_password(password)
    if os.path.exists(filename):
        try:
            df = pd.read_csv(filename)
            new_row = {"username" : username, "password" : hashed_password} #creates key value pair of id password
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index = True)
            df.to_csv(filename, index=False, header=True)
            print(f"User {username} registered successfully")
        except Exception as e:
            print(f"Error registering the user {username}")
    else:
        try:
            df = pd.DataFrame({"username: " [username], "password: " [hashed_password]})
            df.to_csv(filename, index=False)
            print(f"Username {username} registered successfully")
        except Expection as e:
            print(f"Error creating file and registering the user: {e}")

# This is ask the user to enter login and password and invokes task_manager(username) function

def user_login():
    username = input("Please enter username: ")
    password = input("Please enter your password: ")

    if check_username(username):
        try:
            df = pd.read_csv('customer.csv')
            user_row = df[df["username"] == username].iloc[0]
            stored_hashed_password = user_row["password"]
            if hash_password(password) == stored_hashed_password:
                print("Login Successful!")
                print(f"User '{username}' logged in.")  # Print login confirmation
                task_manager(username)  # Call task_manager after login
                print(f"User '{username}' returned from task manager.") # print after the task manager finishes.
            else:
                print("Incorrect Password. Please try again.")
        except Exception as e:
            print(f"Login failed: {e}.")

    else:
        print("Username not found. Would you like to register? (yes/no)")
        choice = input().lower()
        if choice == "yes":
            register_username(username, password)
user_login()
#Task Manager function for users to choose the action 

def task_manager(username):
    while True:
        print("\nTask Manager Options: ")
        print("1. Add Task")
        print("2. View Tasks")
        print("3. Update Task Status")
        print("4. Delete Task")
        print("5: Logout")

        choice = input("Enter your choice: ")

        if choice == "1":
            add_task(username)
        elif choice == "2":
            view_tasks(username)
        elif choice == "3":
            update_task_status(username)
        elif choice == "4":
            delete_task(username)
        elif choice == "5":
            print("Session is logged out")
            break
        else:
            print("Invalid choice.") 

#Function for the user to add task

def add_task(username):
    task = input("Please add your task: ")
    try:
        df = pd.read_csv('taskmanger.csv')
    except FileNotFoundError:
        df= pd.DataFrame(columns=['username', 'task', 'status'])

        existing_task = df[(df['username'] == username) & (df['task'] == task)]
        if not existing_task.empty:
            print("Task already exist for this user.")
            return
             
        with open('taskmanager.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([username, task, "Pending"])
        print(f"Task: {task} added successfully.")    


 #Function for user to view task

def view_tasks(username):
    try:
        df = pd.read_csv('taskmanager.csv')
        user_tasks = df[df['username']==username]
        if not user_tasks.empty:
            print(user_tasks)
        else:
            print("No task found.")
    except FileNotFoundError:
        print(f"Sorry no task found")

#Function for the user to update the task status

def update_task_status(username):
    try:
        df = pd.read_csv('taskmanager.csv')
        user_tasks=df[df['username'] == username]
        if user_tasks.empty:
            print("No tasks found.")
            return
        print(user_tasks[['task', 'status']])   

        task_description = input("Please enter your task description: ")
        match_task = user_tasks[user_tasks['task']== task_description]

        if match_task.empty:
            print("No Task Found.")
            return

        while True:
            new_status = input("Enter the status of task (Complete, Pending or In Progress)")
            if new_status in ["Complete", "Pending", "In Progress"]:
                break
            else:
                print("Invalid status input. Please enter Complete, Pending, or In Progress.")
        task_id = match_task.index[0]
        df.loc[task_id, 'status'] = new_status
        df.to_csv('taskmanager.csv', index=False)
        print("Task status updated.")

    except FileNotFoundError:
        print("No Task Found")

#Deletes existing the task

def delete_task(username):
    """Deletes a task."""
    try:
        df = pd.read_csv('taskmanager.csv')
        user_tasks = df[df['username'] == username]
        if user_tasks.empty:
            print("No tasks found.")
            return

        print(user_tasks)
        task_index = int(input("Enter the index of the task to delete (starting from 0): "))
        if 0 <= task_index < len(user_tasks):
            task_id = user_tasks.index[task_index]
            df = df.drop(task_id)
            df.to_csv('taskmanager.csv', index=False)
            print("Task deleted.")
        else:
            print("Invalid task index.")
    except FileNotFoundError:
        print("No tasks found.")


Please enter username:  himanshi
Please enter your password:  iamgreat


Username not found. Would you like to register? (yes/no)


 yes


User himanshi registered successfully


In [3]:
user_login()

Please enter username:  reema
Please enter your password:  iamgreat


Username not found. Would you like to register? (yes/no)


 yes


User reema registered successfully


In [4]:
user_login()

Please enter username:  himanshi
Please enter your password:  iamgreat


Login Successful!
User 'himanshi' logged in.

Task Manager Options: 
1. Add Task
2. View Tasks
3. Update Task Status
4. Delete Task
5: Logout


Enter your choice:  coding project


Invalid choice.

Task Manager Options: 
1. Add Task
2. View Tasks
3. Update Task Status
4. Delete Task
5: Logout


Enter your choice:  1
Please add your task:  coding project


Task: coding project added successfully.

Task Manager Options: 
1. Add Task
2. View Tasks
3. Update Task Status
4. Delete Task
5: Logout


Enter your choice:  5


Session is logged out
User 'himanshi' returned from task manager.


In [5]:
user_login()

Please enter username:  reema
Please enter your password:  iamgreat


Login Successful!
User 'reema' logged in.

Task Manager Options: 
1. Add Task
2. View Tasks
3. Update Task Status
4. Delete Task
5: Logout


Enter your choice:  1
Please add your task:  maths homework


Task: maths homework added successfully.

Task Manager Options: 
1. Add Task
2. View Tasks
3. Update Task Status
4. Delete Task
5: Logout


Enter your choice:  5


Session is logged out
User 'reema' returned from task manager.
